In [3]:
#Librerías
import pandas as pd
import os
from glob import glob
import datetime
from time import sleep

# Modulos
import utils.scrap_indec as sci
import utils.api_comtrade as com
import utils.analisis as analisis

In [4]:
anio_desde = "2011"
anio_hasta = "2021"
hora_fin_consulta = None #Para evitar problemas con las peticiones a la api de comtrade

### Funciones para el webscrap INDEC

In [5]:
lectura = input("Scrapear INDEC? (tiempo aprox: 1 minuto):")
if lectura.lower()[0] in ["y","s"]:
    df_indec = sci.scrap_base_indec(anio_desde=anio_desde, anio_hasta=anio_hasta, frecuencia="anual")
    df_indec = sci.limpiar_y_corregir_df(df_indec)
    # df_indec.to_csv("../data/ncm indec.csv",index = False)
else: 
    df_indec = pd.read_csv("../data/ncm indec.csv", dtype={"año":str, "ncm":str, "pdes":str})
    print("se importó 'ncm indec.csv' de la carpeta data")

In [6]:
# lectura = input("¿Limpiar carpeta 'descargas'?:")
if lectura.lower()[0] in ["s","y"]:
    for i in glob("../downloads/*", recursive = True): os.remove(i)
    print("Se eliminaron los archivos descargados.")
else:
    print("No se eliminaron los archivos descargados, cuidado con volver a descargarlos")

Se eliminaron los archivos descargados.


### Secreto estadístico en la base del indec

In [7]:
print(sci.plot_df_secreto(df_indec)[0])
sci.plot_df_secreto(df_indec)[1].show()

    año  fob(u$s)_total  fob(u$s)_secreto  proporcion
0  2018    5.800127e+10      3.780257e+09    6.517542
1  2019    6.174265e+10      3.372675e+09    5.462473
2  2020    5.240502e+10      2.478800e+09    4.730080
3  2021    7.390435e+10      4.029968e+09    5.452951


### Análisis de las partidas en base a los datos del INDEC

In [8]:
partida_df = sci.agrupar_por_partida(df_indec)

In [9]:
lectura = input("Leer partidas de comtrade? (tiempo aprox: 4 minutos):")
if lectura.lower()[0] in ["y","s"]:
    print("Va a leerse el número de partidas que reporta Argentina en Comtrade, 4 minutos aproximadamente")
    df_comtrade = com.get_partidas_comtrade(anio_desde=2011, anio_hasta=2021)
    # df_comtrade.to_csv("../data/partidas comtrade.csv",index = False)
    hora_fin_consulta = datetime.datetime.now()
    print(hora_fin_consulta)  
else: 
    df_comtrade = pd.read_csv("../data/partidas comtrade.csv", dtype={"yr":str, "cmdCode":str})
    print("se importó 'partidas comtrade.csv' de la carpeta data")

se importó 'partidas comtrade.csv' de la carpeta data


In [10]:
analisis.chequeo_partidas_com_ind(partida_df, df_comtrade)

Chequeo de los datos para el 2021
Partidas: Indec:  1063 || COMTRADE:  560
Exportaciones: INDEC:  77,934,314,987 || COMTRADE: 77,934,314,724


In [11]:
print(com.plot_secreto_comtrade(df_comtrade)[0])
com.plot_secreto_comtrade(df_comtrade)[1].show()

      yr  TradeValue_total  TradeValue_secreto  proporcion
0   2011       81164548173          1816543118    2.238099
1   2012       77880358762          2102020836    2.699038
2   2013       73843265885          2119715468    2.870560
3   2014       66537271535          1867074570    2.806058
4   2015       55853981342           929970647    1.665003
5   2016       57186982257           692363475    1.210701
6   2017       57483881904           900312563    1.566200
7   2018       49486736260         12071620807   24.393649
8   2019       52570269345         12543857971   23.861126
9   2020       45189097310          9694724791   21.453681
10  2021       52140701130         25793613594   49.469250


# Analisis de partidas
- ¿Cuántas nuevas partidas se exportaron en el período a comparación del 2011?
- ¿Cuántas no se exportaron algún año?
- ¿Cómo evolucionó la participación sobre el comercio mundial de las partidas que se exportaron todos los años del período?

In [12]:
dict_subp_anuales = analisis.get_dict_subp_anuales(partida_df)

print("Número de partidas por año:")   
for año in dict_subp_anuales.keys():    
    print(f"{año}:",len(dict_subp_anuales[año]))

Número de partidas por año:
2011: 1112
2012: 1107
2013: 1099
2014: 1093
2015: 1082
2016: 1079
2017: 1079
2018: 1075
2019: 1084
2020: 1060
2021: 1063


Caen el número de partidas exportadas, lo que va en línea a la hipótesis de concentración de las exportaciones. Pero
- ¿Es afectado por el secreto estadístico?
- ¿Es afectado por el cambio de enmienda?

In [13]:
print("Nuevas partidas a comparación del 2011:")
for año in list(dict_subp_anuales.keys())[1:]:
    print(f"{año}:",len(list(set(dict_subp_anuales[año]) - set(dict_subp_anuales[anio_desde]))))

Nuevas partidas a comparación del 2011:
2012: 28
2013: 25
2014: 28
2015: 28
2016: 23
2017: 25
2018: 29
2019: 32
2020: 22
2021: 24


Hubo nuevas partidas, pero no es un número significativo que invalide el análisis. Hay que ver la evolución de las partidas que se exportaron en todo el período

In [14]:
part_comunes = analisis.get_partidas_comunes(dict_subp_anuales)
len(part_comunes)

968

In [15]:
partida_df = partida_df[partida_df.partida.isin(part_comunes)].reset_index(drop=True)
len(partida_df.partida.unique())

968

Hay 968 partidas que se exportaron todos los años. Pero falta hacer dos filtros:
- Outliers: partidas para las que el reporte de la comtrade sea significativamente mayor a lo que informa el INDEC. (Se eligió 10 millones, pero puede modificarse)
- Relevancia: las partidas deben tener una participación mayor al 0,1% en el comercio internacional. 

In [16]:
comparacion_indec_comtrade = analisis.comparar_indec_comtrade(partida_df, df_comtrade)
comparacion_indec_comtrade.sort_values("diferencia",ascending=True).head(15)

,año,partida,indec,comtrade,diferencia,diferencia en mill
7230,2018,4811,2.204480e+07,8.171296e+07,-59668163.52,-59.668164
9586,2020,8703,3.689599e+08,3.870654e+08,-18105547.86,-18.105548
5859,2017,0805,3.039947e+08,3.191946e+08,-15199936.52,-15.199937
10043,2021,3824,3.170317e+07,4.669585e+07,-14992673.34,-14.992673
8107,2019,3824,3.311100e+07,4.651424e+07,-13403246.68,-13.403247
9075,2020,3824,2.998491e+07,4.198836e+07,-12003457.41,-12.003457
7412,2018,7108,2.232437e+09,2.244301e+09,-11864068.39,-11.864068
8249,2019,5603,3.348150e+07,4.454878e+07,-11067276.69,-11.067277
8940,2020,2827,1.005860e+06,7.638607e+06,-6632746.67,-6.632747
7139,2018,3824,3.820382e+07,4.480414e+07,-6600327.48,-6.600327


In [17]:
partidas_eliminar = comparacion_indec_comtrade[comparacion_indec_comtrade["diferencia en mill"]<=-10].copy().partida.unique()
part_comunes = [x for x in part_comunes if x not in partidas_eliminar]
partidas_eliminar

array(['0805', '4811', '7108', '3824', '5603', '8703'], dtype=object)

Hay algunas partidas con gran diferencia, en especial la "4811". Por lo general, son partidas muy afectadas por el secreto estadístico.

Se eliminan

In [18]:
partida_df = partida_df[~partida_df.partida.isin(partidas_eliminar)].dropna()
len(partida_df.partida.unique())

962

# Participación en el comercio mundial

In [19]:
#HAY ALGUNAS PARTIDAS QUE DICE NO TENER DATOS PERO HAY... creo que es un problema de la api
lectura = input("Leer datos mundiales de comtrade? (tiempo aprox: 3 horas):")
if lectura.lower()[0] in ["y","s"]:
    print("Inicio de pedidos a la api de COMTRADE:",datetime.datetime.now())
    if hora_fin_consulta: 
        sleep(3600)
        print("1 hora de pausa para no saturar a la api")
    df_comtrade_mundo = pd.concat(com.get_datosmundiales(part_comunes, anio_desde, anio_hasta))
    df_comtrade_mundo = com.correccion_descrip_comtrade(df_comtrade_mundo)
    df_comtrade_mundo.yr = df_comtrade_mundo.yr.astype(str)
    df_comtrade_mundo = df_comtrade_mundo.groupby(["yr","cmdCode","cmdDescE"],as_index=False).sum("TradeValue")
    hora_fin_consulta = datetime.datetime.now()
    # df_comtrade_mundo.to_csv(f"../data/comercio mundial comtrade{anio_desde}-{anio_hasta} sin chequear.csv",index=False)
else: 
    df_comtrade_mundo = pd.read_csv("../data/comercio mundial comtrade chequeada.csv", dtype={"yr":str, "cmdCode":str})
    # df_comtrade_mundo = df_comtrade_mundo.groupby(["yr","cmdCode","cmdDescE"],as_index=False).sum("TradeValue")
    print("Se importaron los datos del comercio mundial de 'comercio mundial comtrade.csv'")

Se importaron los datos del comercio mundial de 'comercio mundial comtrade.csv'


In [20]:
# Debería ser una lista vacía
analisis.testeo_datos_mundiales(df_comtrade_mundo)

Si la consulta fue para 2011-2021, todo está correcto. No hay diferencias en las partidas


In [21]:
prop_mundial = analisis.genera_prop_mundial(partida_df,df_comtrade_mundo)

partidas en 2011: 961
partidas en 2012: 961
partidas en 2013: 961
partidas en 2014: 961
partidas en 2015: 961
partidas en 2016: 961
partidas en 2017: 961
partidas en 2018: 961
partidas en 2019: 961
partidas en 2020: 961
partidas en 2021: 961
-----------------
Cantidad de proporciones mayores al 100%: 0


In [22]:
prop_mundial_sin_irrelevantes = analisis.get_partidas_relevantes(prop_mundial)

Cantidad de partidas que caen por debajo del 0,1% del comercio mundial:
2011: 537
2012: 562
2013: 596
2014: 617
2015: 632
2016: 647
2017: 654
2018: 703
2019: 687
2020: 721
2021: 711
----------------------
Partidas irrelevantes: 522
Partidas para análisis: 439


Luego de todos los filtros, quedan 439 partidas para analizar.

In [23]:
for anio in prop_mundial_sin_irrelevantes.año.unique():
    print(len(prop_mundial_sin_irrelevantes.query(f"año == '{anio}'").partida.unique()))

439
439
439
439
439
439
439
439
439
439
439


In [24]:
analisis.plot_evolucion_share(prop_mundial_sin_irrelevantes, anio_desde,anio_hasta)

In [25]:
analisis.plot_evolucion_share_animado(prop_mundial_sin_irrelevantes, anio_desde, anio_hasta)

# Metodología espejo

Para usar esa metodología:
- ¿Existen grandes diferencias a lo que publica el indec?

In [26]:
lectura = input("¿Leer datos espejo de comtrade?: (Dos horas aproximadamente)")
if lectura.lower()[0] in ["y","s"]:
    if hora_fin_consulta: 
        sleep(3600)
        print("1 hora de pausa para no saturar a la api")
    df_comtrade_espejo = com.get_expo_arg_espejo(anio_desde="2011",anio_hasta="2021")
    df_comtrade_espejo = pd.concat(df_comtrade_espejo)
    df_comtrade_espejo = com.correccion_descrip_comtrade(df_comtrade_espejo)
    df_comtrade_espejo.yr = df_comtrade_espejo.yr.astype(str)
    df_comtrade_espejo = df_comtrade_espejo.query('rtTitle != \'ASEAN\' & rtTitle != \'EU\'') #Contabilidad doble
    # df_comtrade_espejo.to_csv("../data/comtrade espejo.csv", index = False)
else: 
    df_comtrade_espejo = pd.read_csv("../data/comtrade espejo.csv", dtype={"cmdCode":str, 'yr':str})

In [27]:
bilateral = analisis.bilateral_xIndec_mEspejoComtrade(df_indec, df_comtrade_espejo)
bilateral

,fob_indec,fob_espejoComtrade,Diferencia en Millones
año,,,
2011,8.298109e+10,83977083361,-995.992079
2012,7.998239e+10,84069689043,-4087.301155
2013,7.596298e+10,80617903668,-4654.922307
2014,6.840435e+10,69513431056,-1109.084438
2015,5.678395e+10,61097934976,-4313.982472
2016,5.790910e+10,60496532840,-2587.435347
2017,5.864473e+10,61970750455,-3326.016807
2018,6.178153e+10,61304585306,476.943295
2019,6.511533e+10,67646833399,-2531.506775


In [28]:
analisis.plot_diferencia_espejo(bilateral)